In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from urllib2 import Request, urlopen, URLError
from bs4 import BeautifulSoup
import re
import turtle
import time
import numpy as np
import math
#import matplotlib.pyplot as plt
from scipy.interpolate import Rbf
#from matplotlib import cm
key = '07d22a108ce3c30cbbda6fe440292e7d'
PI = 3.1415926
classification = [[1,2,3,4],5,[6,7],[8,11],9,10,12,[13,14,16],15,17,18,20]#POI classification
para1 = ['AQI','NO2','PM25','PM10','CO','O3','SO2','Wind','Pressure','Temperature','Humidity']
para2 = ['Wind','Humidity','Pressure','Temperature','NO2','PM25','PM10','CO','O3','SO2','AQI']

In [2]:
def TrafficBasedInterpolation(Coordinate,Data,currentC):
    Dist = list()
    #DIS = 0
    d = 0
    #Dist_copy = list()
    TraffiC = np.asarray(getTrafficFromGPS(currentC[0],currentC[1],1000)[0])
    for cor in Coordinate:
        #print cor
        if getTrafficFromGPS(cor[0],cor[1],1000):
            TraffiV = np.asarray(getTrafficFromGPS(cor[0],cor[1],1000)[0])
        dis = (np.sqrt(sum((TraffiC - TraffiV)**2)))**(-1)
        Dist.append(dis)
        #DIS = DIS + dis
    d = Dist.index(min(Dist))
    return d

def POIBasedInterpolation(Coordinate,Data,currentC):
    Dist = list()
    #DIS = 0
    d = 0
    #Dist_copy = list()
    POIC = np.asarray(getPOIsFromGPS(currentC[0],currentC[1],1000))
    for cor in Coordinate:
        #print cor
        POI = np.asarray(getPOIsFromGPS(cor[0],cor[1],1000))
        dis = (np.sqrt(sum((POIC - POI)**2)))**(-1)
        Dist.append(dis)
        #DIS = DIS + dis
    d = Dist.index(min(Dist))
    return d

def getOnlineDataFromMS_Station(Coordinate):
    Data = list()
    for i in range(0,11):
        l = list()
        Data.append(l)
    for cor in Coordinate:
        URL = 'http://urbanair.msra.cn/U_Air/SearchStation?Latitude='+str(cor[0])+'&Longitude='+str(cor[1])+'&Standard=0'
        request = Request(URL)
        try:
            response = urlopen(request)
            kittens = response.read()
            llist = re.split(':',kittens);
            #print kittens
            for i in range(3,14):
                a = re.split(',',llist[i])
                if a[0] == 'null':
                #Data[i-1].append(None)
                    Data[i-3].append(None)
                else:
                    #Data[i-1].append(float(a[0]))
                    Data[i-3].append(float(a[0]))
                #print type(kittens)
        except URLError, e:
            print 'Got an error code:', e
    return Data

def getOnlineDataFromMS(Coordinate):
    Data = list()
    for i in range(0,11):
        l = list()
        Data.append(l)
    for cor in Coordinate:
        URL = 'http://urbanair.msra.cn/U_Air/SearchGeoPoint?Latitude='+str(cor[0])+'&Longitude='+str(cor[1])+'&Standard=0&Culture=zh-CN'
        request = Request(URL)
        try:
            response = urlopen(request)
            kittens = response.read()
            llist = re.split(':',kittens);
            #print kittens
            for i in range(1,8):
                a = re.split(',',llist[i])
                if a[0] == 'null':
                    Data[i-1].append(None)
                else:
                    Data[i-1].append(float(a[0]))
                #print type(kittens)
            for i in range(10,14):
                a = re.split(',',llist[i])
                if a[0] == 'null':
                    Data[i-3].append(None)
                else:
                    Data[i-3].append(float(a[0]))
                #print type(kittens)
        except URLError, e:
            print 'Got an error code:', e
    return Data

def getPredictPM25FromGPS(x,y):
    URL = 'http://urbanair.msra.cn/U_Air/SearchPrediction_48V2?Latitude=' + str(x) + '&Longitude=' + str(y) + '&Pollutant=PM25' 
    request = Request(URL)
    predict = []
    try:
        response = urlopen(request)
        kittens = response.read()
        llist = re.split(':',kittens);
        llist = re.split(',',llist[1])
        predict.append(int(llist[0][1:]))
        for i in range(1,13):
            predict.append(int(llist[i]))
        predict.append(int(llist[13][:-1]))
    except URLError, e:
        print 'Got an error code:', e
    return predict#[0:7定时预测，8:13三个区间]

def getTrafficFromGPS(x,y,r):
    URL = 'http://restapi.amap.com/v3/traffic/status/circle?key='+key+'&location='+str(y)+','+str(x)+'&radius='+str(r)+'&output=xml&offset=20&page=1&extensions=all' 
    request = Request(URL)
    try:
        response = urlopen(request)
        kittens = response.read()
    except URLError, e:
        print 'Got an error code:', e
    soup = BeautifulSoup(kittens,"html.parser")
    trafficV = np.zeros(4)
    #print type(soup.evaluation.contents)
    if soup.evaluation==None:
        print("No Data")
        return False
    if soup.evaluation.expedite.contents==[]:
        print ("No Data")
        return False
    #print soup.evaluation.congested.contents
    #print soup.evaluation.blocked.contents
    #print soup.evaluation.unknown.contents
    trafficV[0] = float(soup.evaluation.expedite.contents[0][:4])
    trafficV[1] = float(soup.evaluation.congested.contents[0][:4])
    trafficV[2] = float(soup.evaluation.blocked.contents[0][:4])
    trafficV[3] = float(soup.evaluation.unknown.contents[0][:4])
    return trafficV,True

def getPOIsFromGPS(x,y,r):
    URL = 'http://restapi.amap.com/v3/place/around?key='+key+'&location='+str(y)+','+str(x)+'&output=xml&radius='+str(r)+'&offset=20&page=1&extensions=all'
    request = Request(URL)
    try:
        response = urlopen(request)
        kittens = response.read()
    except URLError, e:
        print 'Got an error code:', e
    soup = BeautifulSoup(kittens,"html.parser")
    a = soup.pois
    poiV = np.zeros(12)
    for child in a:
        #print child.typecode.contents[0][:6]
        b = int(child.typecode.contents[0][:6])
        if b/10000>0 and b/10000<5:
            poiV[0] = poiV[0]+1
        elif b/10000==5:
            poiV[1] = poiV[1]+1
        elif b/10000==6 or b/10000==7:
            poiV[2] = poiV[2]+1
        elif b/10000==8 or b/10000==11:
            poiV[3] = poiV[3]+1
        elif b/10000==9:
            poiV[4] = poiV[4]+1
        elif b/10000==10:
            poiV[5] = poiV[5]+1
        elif b/10000==12:
            poiV[6] = poiV[6]+1
        elif b/10000==13 or b/10000==14 or b/10000==16:
            poiV[7] = poiV[7]+1
        elif b/10000==15:
            poiV[5] = poiV[8]+1
        elif b/10000==17:
            poiV[5] = poiV[9]+1
        elif b/10000==18:
            poiV[5] = poiV[10]+1
        elif b/10000==20:
            poiV[5] = poiV[11]+1
    return poiV

def getLocationFromKeywords(keywords):
    URL = 'http://restapi.amap.com/v3/place/text?&keywords=' + keywords + '&city=beijing&output=xml&offset=20&page=1&key='+key+'&extensions=all'
    request = Request(URL)
    try:
        response = urlopen(request)
        kittens = response.read()
    except URLError, e:
        print 'Got an error code:', e
    soup = BeautifulSoup(kittens,"html.parser")
    Str = soup.location.string
    lstr = re.split(',',Str)
    lon = float(lstr[0])
    lat = float(lstr[1])
    return lat,lon

def DistantBasedInterpolation(Coordinate,Data,currentC):
    Dist = list()
    DIS = 0
    d = 0
    #Dist_copy = list()
    for cor in Coordinate:
        dis = (np.sqrt(((cor[0] - currentC[0])**2+(cor[1]-currentC[1])**2)))**(-1)
        Dist.append(dis)
        DIS = DIS + dis
    for i in range(0,len(Coordinate)):
        #print Dist[i]
        if Data[i]==None:
            Data[i] = 0
        d = d + (Dist[i]/DIS)*Data[i]
    return d

def compare(a,b):
    x=[a[0]-p0[0],a[1]-p0[1]]
    y=[b[0]-p0[0],b[1]-p0[1]]
    dx=(x[0]**2+x[1]**2)**0.5
    dy=(y[0]**2+y[1]**2)**0.5
    cosa=x[0]/dx
    cosb=y[0]/dy
    if cosa < cosb:
        return 1
    elif cosa > cosb:
        return -1
    else:
        if dx<dy:
            return -1
        elif dx>dy:
            return 1
        else:
            return 0
        
def windingNumber(p, poly):
    sum = 0
    px = p[0]
    py = p[1]
    print px,py
    l = len(poly)
    for i in range(0,l):
        if i == 0:
            j = l-1
        else:
            j = i-1
        sx = poly[i][0]
        sy = poly[i][1]
        tx = poly[j][0]
        ty = poly[j][1]
        #点与多边形顶点重合或在多边形的边上
        if (sx - px) * (px - tx) >= 0 and (sy - py) * (py - ty) >= 0 and (px - sx) * (ty - sy) == (py - sy) * (tx - sx):
            return True
        #点与相邻顶点连线的夹角
        angle = math.atan2(sy - py, sx - px) - math.atan2(ty - py, tx - px)
        #确保夹角不超出取值范围（-π 到 π）
        if angle >= PI:
            angle = angle - PI * 2
        elif angle <= -PI: 
            angle = angle + PI * 2
        sum =sum + angle
        #计算回转数并判断点和多边形的几何关系
        if sum / PI >= -0.01 and sum / PI <= 0.01:
            return False
        else:
            return True
        
def ScatterInterpolation(Coordinate,currentC,Data):
    x = []
    y = []
    for i in range(0,len(Coordinate)):
        x.append(Coordinate[i][0])
        y.append(Coordinate[i][1])
    x = np.asarray(x)
    y = np.asarray(y)
    z = np.asarray(Data)
    rbf = Rbf(x, y, z, epsilon=1)
    ZI = rbf(currentC[0], currentC[1])
    return ZI

def TriangleInterpolation(x1,y1,x2,y2,x3,y3,x,y,a,b,c):
    d = a + (b-a)*((x-x1)*(y3-y1)+(y-y1)*(x1-x3))/((y3-y1)*(x2-x1)+(y1-y2)*(x3-x1)) +\
    (c-a)*((y-y1)*(x2-x1)+(y1-y2)*(x-x1))/((y3-y1)*(x2-x1)+(y1-y2)*(x3-x1))
    return d   

def FindThreePoints(Coordinate,currentC):
    Dist = list()
    #Dist_copy = list()
    for cor in Coordinate:
        dis = np.sqrt(((cor[0] - currentC[0])**2+(cor[1]-currentC[1])**2))
        Dist.append(dis)
        #Dist_copy.append(dis)
    MAX = max(Dist)
    p1 = Dist.index(min(Dist))
    Dist[p1] = MAX
    p2 = Dist.index(min(Dist))
    Dist[p2] = MAX
    p3 = Dist.index(min(Dist))
    #return Dist_copy[p1],Dist_copy[p2],Dist_copy[p3]
    return Coordinate[p1],Coordinate[p2],Coordinate[p3]

def getCoordinate():
    file_ads = "/Users/Claire/Desktop/intern/20170317/UrbanAir_get/airStation.txt" 
    f = open(file_ads,"r");
    line = f.readline();
    point = list()
    Coordinate = list()
    while line:
    #print line
        llist = re.split('=',line);
        x = re.split('&',llist[1])
        y = re.split('&',llist[2])
        cor = [float(x[0]),float(y[0])]
        point.append(cor)
        Coordinate.append(cor)
        line = f.readline();
    return point, Coordinate

def getCircle(point):
    global p0
    n=0
    for i in range(len(point)):
        if point[n][1]>point[i][1]:
            n=i
    p0=point[n]
    point.pop(n)
    point.sort(compare)
    point.insert(0,p0)
    ep=point[:]#复制元素，操作ep不会对point产生影响
    tag=0
    while tag==0:
        tag=1
        l=len(ep)
        for i in range(l):
            i1,i2,i3=(i,(i+1)%l,(i+2)%l)
            x,y,z=(ep[i1],ep[i2],ep[i3])
            a1,a2=((y[0]-x[0],y[1]-x[1]),(z[0]-y[0],z[1]-y[1]))
            if a1[0]*a2[1]-a1[1]*a2[0] < 0:
                tag=0
                ep.pop(i2)
                break
            elif a1[0]*a2[1]-a1[1]*a2[0]==0 and a1[0]*a2[0]<0:
            #==0应改写,360度的情况
                tag=0
                ep.pop(i2)
                break
    return ep;

def getDataForKeywords(keywords):
    point,Coordinate = getCoordinate()
    x,y = getLocationFromKeywords(keywords)
    p = [x,y] 
    ep = getCircle(point)
    Data = getOnlineDataFromMS(Coordinate)
    if windingNumber(p,ep):
        d = DistantBasedInterpolation(Coordinate,Data[0],p)
        dPOI = POIBasedInterpolation(Coordinate,Data[0],p)
        dTraffic = TrafficBasedInterpolation(Coordinate,Data,p)
    else:
        dPOI = POIBasedInterpolation(Coordinate,Data[0],p)
    return d,dPOI,dTraffic

In [3]:
#TrafficBasedInterpolation(Coordinate,Data,currentC) #匹配最相近的交通状况
#POIBasedInterpolation(Coordinate,Data,currentC) #匹配最相近的POI状况
#Data = getOnlineDataFromMS(Coordinate) #得到基站的时实时数据，数据类型详见para
#getPredictPM25FromGPS(x,y) #给定坐标得到PM25预测数据
#getTrafficFromGPS(x,y,r) #得到交通数据通过坐标及半径
#getPOIsFromGPS(x,y,r) #得到POI数据通过坐标和半径
#getLocationFromKeywords(keywords) #通过关键字找到坐标
#DistantBasedInterpolation(Coordinate,Data,currentC) #通过距离插值得到某个点的预测
#windingNumber(p, poly) #判断点p是否在poly多边形内部
#point, Coordinate = getCoordinate() #得到基站的坐标
#ep = getCircle(point) #得到点的外包多边形

In [4]:
point, Coordinate = getCoordinate()
#ep = getCircle(point)
Data_all = getOnlineDataFromMS_Station(Coordinate)
Data = Data_all[10]
xn = 10
yn = 10
xmin,ymax = getLocationFromKeywords("亦庄开发区")
xmax,ymin = getLocationFromKeywords("北京植物园")
x_gap = (xmax-xmin)/xn
y_gap = (ymax-ymin)/yn
for i in range(0,xn):
    for j in range(0,yn):
        xc = xmin + x_gap * i
        yc = ymin + y_gap * j
        pc = [xc,yc]
        d = DistantBasedInterpolation(Coordinate,Data,pc)
        Coordinate.append(pc)
        Data.append(d)

In [5]:
X_input = []
for cor in Coordinate:
    x = cor[0]
    y = cor[1]
    r = 3000
    poiV = list(getPOIsFromGPS(x,y,r))
    TraffiV = getTrafficFromGPS(x,y,r)
    if TraffiV:
        Tr = list(TraffiV[0])
    else:
        Tr = [0,0,0,0]
    poiV.extend(Tr)
    X_input.append(poiV)
    
X_input = np.asarray(X_input)
y_input = np.asarray(Data)
lr = LinearRegression()
lr.fit(X_input,y_input)
#lr.intercept_,lr.coef_
y_pred = lr.predict(X_input)

No Data
No Data
No Data


In [8]:
sum(y_pred[36:])

27719.007839866554

In [9]:
sum(y_input[36:])

27700.081326400777

In [7]:
y_pred[36:]

array([ 279.63041969,  280.11408612,  282.92878065,  278.30856022,
        278.80387954,  279.423809  ,  275.90737416,  275.01106462,
        281.79556325,  276.10798392,  286.27299024,  276.70951087,
        278.1654621 ,  278.21649406,  278.60113896,  279.0821823 ,
        280.37392296,  275.26542491,  278.14723584,  275.69754876,
        273.34716698,  276.28082967,  279.2374776 ,  276.51357933,
        280.19527244,  276.15865513,  275.50050361,  275.24484079,
        274.17068976,  277.87307173,  274.32878103,  278.13321869,
        277.13383105,  277.1415091 ,  278.07581154,  278.42873107,
        278.03637401,  276.51683631,  278.02224843,  276.45332373,
        276.77431369,  276.86039974,  277.43994973,  276.36607683,
        277.10469093,  276.34863518,  276.18607571,  275.68005644,
        277.60722521,  277.61699786,  277.94864152,  274.16132796,
        277.52706755,  277.56861666,  277.42697656,  276.95386698,
        280.22315858,  277.46157963,  276.63970857,  276.42953

In [6]:
y_input[36:]

array([ 282.08661869,  280.45057467,  279.20122151,  278.02632526,
        276.80571987,  275.63689856,  274.84557746,  274.53248577,
        274.42730716,  273.93199215,  281.18458885,  279.78130222,
        278.75936221,  277.78240283,  276.71475202,  275.61411582,
        275.06262365,  274.99774326,  274.95696801,  274.70234021,
        280.35810407,  279.31363468,  278.50964662,  277.66808672,
        276.64135033,  275.0417157 ,  274.76948768,  275.16812912,
        275.30288286,  275.24544878,  279.7325224 ,  278.94936265,
        278.45224101,  277.63138201,  276.70787231,  273.78412825,
        274.15272171,  275.24038828,  275.56144271,  275.57794196,
        279.37455296,  278.65250653,  278.13717482,  277.47012604,
        276.86308051,  277.18910453,  273.02162489,  275.81525457,
        275.88093082,  275.85529067,  279.2742269 ,  278.43537773,
        277.88203   ,  277.33341272,  276.82081621,  276.44809718,
        276.70723157,  276.60914799,  276.28018776,  276.12854

In [ ]:
#等高线图
import matplotlib.pyplot as plt 
x = range(0,xn)
y = range(0,yn)
z = y_pred[36:]
z = np.reshape(z,(10,10))
z = np.reshape(z,(10,10))
fig = plt.figure()
#2行2列的子图中的第一个，第一行的第一列
subfig1 = fig.add_subplot(1,1,1)
#画等值线云图
surf1 = plt.contourf(x, y, z)
#添加色标
fig.colorbar(surf1)
#添加标题
plt.title('contourf+colorbar')
plt.show()

In [ ]:
#灰度图
im = z
plt.subplot(111); plt.imshow(im, cmap='gray')
plt.show()

In [4]:
keywords = "北京植物园"
x,y = getLocationFromKeywords(keywords)

In [5]:
x

40.001403

In [6]:
y

116.209714